# [최종] 실시간 급등 주식 확인 후, 뉴스 알려주는 slack bot

In [1]:
import slack_sdk
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import schedule
import time

# 슬랙 토큰 및 클라이언트 초기화 
slack_token = "개인 슬랙 토큰을 입력해주세요"
client = slack_sdk.WebClient(token=slack_token)

options = Options()
options.add_argument("--headless") 

def fetch_and_send_to_slack():
    browser1 = webdriver.Chrome(options = options)
    url1 = "https://finance.naver.com/sise/sise_rise.naver" #급등주 10개 추출
    browser1.get(url1)

    titles = browser1.find_elements(By.CLASS_NAME, 'tltle')
    rises = browser1.find_elements(By.CLASS_NAME, 'tah.p11.red01')

    stock_list = []
    title_list = []
    message = "종목\t\t상승\n"
    for title, rise in zip(titles[:10], rises[:10]):  # 종목 10개 가져옴
        title_text = title.text
        title_list.append(title_text)
        rise_value = rise.text
        stock = title.get_attribute('href')[-6:]  # 종목번호
        stock_list.append(stock)
        message += f"{title_text}\t\t{rise_value}\n"
    # browser1.close()

    for stock, title in zip(stock_list, title_list):
        browser2 = webdriver.Chrome(options = options)
        url2 = f"https://finance.naver.com/item/news.naver?code={stock}" #급둥주 10개 추출한 데이터에서 각 뉴스 자료
        browser2.get(url2)

        iframe = browser2.find_element(By.ID, 'news_frame')
        browser2.switch_to.frame(iframe)

        news_titles = browser2.find_elements(By.CLASS_NAME, "tit")
        dates = browser2.find_elements(By.CLASS_NAME, "date")
        message += "---------------------------------------------------------------------------------------------\n"
        message += f"'{title}' 주요 뉴스\n\n제목\t\t\t\t\t\t\t\t날짜\n"
        for news_title, date in zip(news_titles, dates):
            title_text = news_title.text
            date_text = date.text
            message += f"{title_text}\t\t\t\t{date_text}\n"

        browser2.switch_to.default_content()
        # browser2.close()

    # 슬랙에 메시지 전송
    client.chat_postMessage(channel='stock급등-알림', text=message)

# 스케줄러 설정
schedule.every(10).minutes.do(fetch_and_send_to_slack)

# 스케줄러 루프
while True:
    schedule.run_pending()
    time.sleep(1)


# 각각의 실행 과정 코드 

# 실시간 급등 종목 추출

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#종목 10개 가져옴
count = 10 #종목 개수
browser1 = webdriver.Chrome()

url1 = "https://finance.naver.com/sise/sise_rise.naver"
browser1.get(url1)

titles = browser1.find_elements(By.CLASS_NAME, 'tltle')
rises = browser1.find_elements(By.CLASS_NAME, 'tah.p11.red01')

stock_list =[]
title_list = []
print("종목\t\t상승")
for title, rise in zip(titles[: count], rises[: count]):
    title_text = title.text
    title_list.append(title_text)
    rise_value = rise.text
    stock = title.get_attribute('href')[-6 :] #종목번호
    stock_list.append(stock)
    print(f"{title_text}\t\t{rise_value}")




#bs4로 보니 class='tit'이 보이지 않음 -> iframe으로 되어 있음

# 실시간 급등 주식 뉴스가져오기

In [ ]:
for stock, title in zip(stock_list, title_list):    
    browser2 = webdriver.Chrome()
    url2 = f"https://finance.naver.com/item/news.naver?code={stock}"
    browser2.get(url2)

    # id가 'news_frame'인 iframe 찾기
    iframe = browser2.find_element(By.ID, 'news_frame')

    # iframe으로 컨텍스트 전환
    browser2.switch_to.frame(iframe)

    # iframe 내에서 필요한 작업 수행
    # 예: iframe 내의 모든 텍스트를 출력
    titles = browser2.find_elements(By.CLASS_NAME, "tit")
    dates = browser2.find_elements(By.CLASS_NAME, "date")
    print("---------------------------------------------------------------------------------------------")
    print(f"{title} 주요 뉴스")
    print()
    print("제목\t\t\t\t\t\t\t\t날짜")
    for title, date in zip(titles, dates):
        title_text = title.text
        date_text = date.text
        print(f"{title_text}\t\t\t\t{date_text}")
        
    browser2.switch_to.default_content()

    browser2.close()

# slack API연동 방법

In [ ]:
import slack_sdk

#내 Bot User OAuth Token
slack_token = "개인 슬랙 토큰을 입력해주세요"
client = slack_sdk.WebClient(token = slack_token)

client.chat_postMessage(channel = '일반', text = 'Hello world!')